In [1]:
from nutrition_data.nutrition_data_loader import NutritionDataLoader
from loose_limits_algorithm.loose_limits_algorithm import LooseLimitsAlgorithm
from random import randint
from product import Solution
from main import Model
from selection_algos import *

import numpy as np



In [2]:
DATA_FILE = "./nutrition_data/complete_database.csv"
PRODUCTS_NUMBER_RANGE = (5, 15)
MIN_GRAMS = 50
MAX_GRAMS = 2000

CATEGORIES = [
    "Vegetables",
    "Meats",
    "Dairy and Egg Products",
    "Fats and Oils",
    "Grains and Pasta",
    "Fruits"
]

restrictions = {
    "calories": (2100, 2500),
    "carbs": (260, 320),
    "protein": (65, 80),
    "fat": (70, 90)
}

avg_restrictions = {
    name: (r[0] + r[1]) / 2 for name, r in restrictions.items()
}

def simple_looseness(r_d, r_u, x):
    if x < r_d:
        return np.exp(min((r_d - x)/r_d, 2)) - 1
    elif x <= r_u:
        return 0
    else:
        return np.exp(min((x - r_u)/r_u, 2)) - 1

looseness_functions = {
    "calories": simple_looseness,
    "carbs": simple_looseness,
    "protein": simple_looseness,
    "fat": simple_looseness
}

products_data_loader = NutritionDataLoader(DATA_FILE)
products_data_loader.initial_preprocessing()
model = Model(products_data_loader, restrictions)

algo = LooseLimitsAlgorithm(restrictions, looseness_functions, 100, MIN_GRAMS, MAX_GRAMS)
solutions = [
    algo.look_for_solution(products_data_loader.generate_products(CATEGORIES, [3, 2, 2, 2, 2, 2])) for _ in range(1000)
]

print("Correct solutions:")
start_solutions = list(filter(lambda t: t[1] == 0, solutions))
print(len(start_solutions))
print()

Correct solutions:
37



In [17]:
start_solutions = [s[0] for s in start_solutions]

In [95]:
import importlib
import selection_algos
from solution_scoring import solution_score
importlib.reload(selection_algos)

rank = selection_algos.selection_ranking(start_solutions, 5)
tour = selection_algos.selection_tournament(start_solutions, 5)
roul = selection_algos.selection_roulette(start_solutions, 5)
print(list(map(solution_score, rank)))
print(list(map(solution_score, tour)))
print(list(map(solution_score, roul)))

[310.00033546290683, 298.0, 295.0000045206588, 339.00000083152884, 368.0000453999298]
[412.00000000001387, 310.00033546290683, 386.0000003059023, 295.0000045206588, 298.0]
[295.0000045206588, 298.0, 520.0000453999437, 642.0000022603294, 386.0000003059023]
